In [32]:
import numpy as np
from classifiers import *
from data_generation import *
from timeit import default_timer as timer

# Load Dataset

In [49]:
train_path = 'datasets/mnist_train.csv'
test_path = 'datasets/mnist_test.csv'
y_col = 0

In [15]:
def get_features_and_labels(dataset, y_col):
    y = dataset[:, y_col:y_col+1]
    selector = [col for col in range(dataset.shape[1]) if col != y_col]
    X = dataset[:, selector]
    return X, y

In [16]:
## Import data and get sets
train_set = np.genfromtxt(train_path, delimiter=',')
test_set = np.genfromtxt(test_path, delimiter=',')

train_X, train_y = get_features_and_labels(train_set, y_col)
test_X, test_y = get_features_and_labels(test_set, y_col)

In [17]:
# Optional: Truncate datasets/mess around with them as needed
n = 1000
train_X, train_y = train_X[0:n, :], train_y[0:n, :]
test_X, test_y = test_X[0:n, :], test_y[0:n, :]

#Normalize Data
train_X, train_y = train_X - np.mean(train_X) , train_y
test_X, test_y = test_X - np.mean(test_X) , test_y

In [18]:
n = test_y.size
train_y = train_y.astype(int)
test_y = test_y.astype(int)

# Run Quantile Regression 

In [31]:
def train_classifiers(X_train, y_train, X_test, y_test, a, reg_param=1,
                      kernel='linear', kernel_param=1, loss_function='logistic',
                      opt_type='SGD', opt_params={'learning_rate': 1}):
    results = {}
    
    base = opt_params['plot_file']
    opt_params['plot_file'] = 'IT' + base
    start = timer()
    clf5 = QuantileIT(gamma=a, alpha=reg_param, kernel_type=kernel, opt_type=opt_type, opt_params=opt_params,
                              kernel_param=kernel_param, loss_function=loss_function)
    clf5.fit(X_train, y_train)
    preds_5 = clf5.predict(X_test)
    end = timer()
    
    abs_loss = weighted_absolute_loss(preds_5, y_test, a)
    print('Weighted Absolute Loss of QuantileIT, gamma=' + str(a) + ' %s' %
          abs_loss)
    
    preds_5_in = clf5.predict(X_train)
    abs_loss_in = weighted_absolute_loss(preds_5_in, y_train, a)
    print('In Sample Weighted Absolute Loss of QuantileIT, gamma=' + str(a) + ' %s' %
          abs_loss_in)
    print('Time Elapsed: ' + str(end - start))
    
    results[clf5] = {'AbsLoss': abs_loss, 'Preds': preds_5}
    
    print(' ')

    opt_params['plot_file'] = 'AT' + base
    start = timer()
    clf6 = QuantileAT(gamma=a, alpha=reg_param, kernel_type=kernel, opt_type=opt_type, opt_params=opt_params,
                              kernel_param=kernel_param, loss_function=loss_function)
    clf6.fit(X_train, y_train)
    preds_6 = clf6.predict(X_test)
    end = timer()
    abs_loss = weighted_absolute_loss(preds_6, y_test, a)
    print('Weighted Absolute Loss of QuantileAT, gamma=' + str(a) + ' %s' %
          abs_loss)
    
    preds_6_in = clf6.predict(X_train)
    print('In Sample Weighted Absolute Loss of QuantileAT, gamma=' + str(a) + ' %s' %
          abs_loss_in)
    print('Time Elapsed: ' + str(end - start))
    
    results[clf6] = {'AbsLoss': abs_loss, 'Preds': preds_6}
    return results

In [41]:
reg = 10
kern = 'linear'
kern_param = 1
lf = 'hinge'

parms = {'learning_rate': 0.00000001, 'momentum_gamma': 0.9,
         'batch_size': 500, 'plot_file': 'test.png'}

results = train_classifiers(train_X, train_y, test_X, test_y, 0.3, reg_param=reg,
                            kernel=kern, kernel_param=kern_param, loss_function=lf,
                            opt_type='SGD', opt_params=parms)

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Generating plot ITtest.png
Weighted Absolute Loss of QuantileIT, gamma=0.3 0.7313
In Sample Weighted Absolute Loss of QuantileIT, gamma=0.3 0.3651999999999999
Time Elapsed: 50.55259421400024
 
Generating plot ATtest.png
Weighted Absolute Loss of QuantileAT, gamma=0.3 0.7626999999999999
In Sample Weighted Absolute Loss of QuantileAT, gamma=0.3 0.3651999999999999
Time Elapsed: 50.26658562400007


In [42]:
for clf in results.keys():
    print(results[clf]['Preds'])
    print(clf.theta_)

[8 3 0 2 4 0 5 6 0 9 0 0 7 1 4 3 7 5 0 4 8 5 3 5 2 0 7 3 0 1 5 3 6 4 5 0 5
 2 2 2 0 7 7 0 6 2 5 2 7 4 4 3 0 3 0 5 0 4 7 5 7 6 7 6 7 6 0 2 4 0 5 0 0 7
 3 5 2 3 8 8 7 5 3 8 6 5 6 5 6 0 2 5 5 5 3 2 3 6 2 6 4 0 7 4 1 7 0 0 8 4 6
 5 3 8 4 6 6 4 7 6 4 4 6 5 7 6 3 3 9 4 5 4 2 2 7 0 0 2 4 6 6 6 5 1 6 0 5 0
 0 4 6 3 3 5 3 4 4 0 0 4 6 5 5 5 6 4 4 0 0 5 5 5 0 3 6 6 0 4 0 3 0 6 5 0 5
 6 0 5 0 1 2 4 0 8 0 3 2 6 7 5 4 1 0 2 0 3 9 5 3 7 4 5 5 5 9 2 4 0 5 0 9 0
 3 7 0 0 5 5 0 5 4 3 6 7 5 6 5 5 5 1 5 6 7 6 5 4 0 5 4 0 7 2 8 5 5 6 0 2 0
 0 6 2 7 7 3 1 5 1 0 0 5 0 2 6 6 6 2 5 6 2 3 7 4 5 6 3 6 0 0 5 0 3 7 4 0 6
 0 2 0 4 1 3 0 5 5 0 4 7 5 3 0 0 0 4 2 5 5 4 0 0 7 7 8 2 4 6 4 0 5 2 3 0 2
 3 0 4 5 5 2 4 3 5 0 1 0 0 3 4 0 8 4 5 2 5 2 6 5 1 6 6 3 4 4 0 0 0 0 5 6 3
 8 0 0 5 6 4 7 1 2 3 0 5 2 7 5 0 5 0 2 7 0 3 5 1 6 5 3 4 3 5 0 6 4 5 3 3 5
 0 2 4 8 7 9 8 4 7 2 6 5 2 3 0 8 3 0 5 8 1 5 4 0 4 4 6 7 5 1 6 8 7 0 0 4 4
 4 0 5 6 7 4 4 1 1 0 6 0 0 5 8 0 8 6 4 1 4 9 4 4 3 6 3 8 3 0 7 5 2 3 4 6 0
 7 6 4 5 0 7 6 6 0 4 5 2 

# Run Multiclass Test
Learning $3 \lg k$ quantiles

In [89]:
def mode(array):
    most = max(list(map(array.count, array)))
    return list(set(filter(lambda x: array.count(x) == most, array)))

In [90]:
ATs = []
AT_predictions = []
s = int(3 * np.log2(np.unique(train_y).size)) # Number of quantiles
for i in range(1, s):
    a = i/s

    clf6 = QuantileAT(gamma=a, alpha=1., kernel_type='linear', kernel_param=1, loss_function='logistic')
    clf6.fit(train_X, train_y)
    ATpredictions = clf6.predict(test_X)
    print('Weighted Loss of QuantileAT, gamma=' + str(a) + ' %s' %
          weighted_absolute_loss(ATpredictions, test_y, a))
    print('0-1 Loss of QuantileAT, gamma=' + str(a) + ' %s' %
          metrics.zero_one_loss(ATpredictions, test_y))
    ATs.append(clf6)
    AT_predictions.append(ATpredictions)

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Weighted Loss of LogisticQuantileAT, gamma=0.1111111111111111 0.4457777777777777
0-1 Loss of LogisticQuantileAT, gamma=0.1111111111111111 0.802
Weighted Loss of LogisticQuantileAT, gamma=0.2222222222222222 0.6322222222222221
0-1 Loss of LogisticQuantileAT, gamma=0.2222222222222222 0.78
Weighted Loss of LogisticQuantileAT, gamma=0.3333333333333333 0.7686666666666667
0-1 Loss of LogisticQuantileAT, gamma=0.3333333333333333 0.7969999999999999
Weighted Loss of LogisticQuantileAT, gamma=0.4444444444444444 0.8154444444444444
0-1 Loss of LogisticQuantileAT, gamma=0.4444444444444444 0.794
Weighted Loss of LogisticQuantileAT, gamma=0.5555555555555556 0.8216666666666668
0-1 Loss of LogisticQuantileAT, gamma=0.5555555555555556 0.8109999999999999
Weighted Loss of LogisticQuantileAT, gamma=0.6666666666666666 0.7786666666666667
0-1 Loss of LogisticQuantileAT, gamma=0.6666666666666666 0.839
Weighted Loss of LogisticQuantileAT, gamma=0.7777777777777778 0.665
0-1 Loss of LogisticQuantileAT, gamma=0.777

In [91]:
AT_preds = np.zeros((n, s))
for i in range(1, s):
    AT_preds[:, i] = AT_predictions[i - 1]
AT_preds = AT_preds.astype(int)
quantile_predictions = np.zeros((n, 1))

for i in range(1, n):
    quantiles = AT_preds[i, :]
    quantile_predictions[i, :] = np.random.choice(mode(quantiles.tolist()))
    
metrics.zero_one_loss(quantile_predictions, test_y)

0.808

In [92]:
ATs = []
AT_predictions = []
s = int(3 * np.log2(np.unique(train_y).size)) # Number of quantiles
for i in range(1, s):
    a = i/s

    clf6 = QuantileAT(gamma=a, alpha=1., kernel_type='rbf', kernel_param=1, loss_function='logistic')
    clf6.fit(train_X, train_y)
    ATpredictions = clf6.predict(test_X)
    print('Weighted Loss of QuantileAT, gamma=' + str(a) + ' %s' %
          weighted_absolute_loss(ATpredictions, test_y, a))
    print('0-1 Loss of QuantileAT, gamma=' + str(a) + ' %s' %
          metrics.zero_one_loss(ATpredictions, test_y))
    ATs.append(clf6)
    AT_predictions.append(ATpredictions)

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Weighted Loss of LogisticQuantileAT, gamma=0.1111111111111111 0.4546666666666666
0-1 Loss of LogisticQuantileAT, gamma=0.1111111111111111 0.874
Weighted Loss of LogisticQuantileAT, gamma=0.2222222222222222 0.8131111111111111
0-1 Loss of LogisticQuantileAT, gamma=0.2222222222222222 0.884
Weighted Loss of LogisticQuantileAT, gamma=0.3333333333333333 1.0716666666666665
0-1 Loss of LogisticQuantileAT, gamma=0.3333333333333333 0.884
Weighted Loss of LogisticQuantileAT, gamma=0.4444444444444444 1.2127777777777777
0-1 Loss of LogisticQuantileAT, gamma=0.4444444444444444 0.893
Weighted Loss of LogisticQuantileAT, gamma=0.5555555555555556 1.3025555555555557
0-1 Loss of LogisticQuantileAT, gamma=0.5555555555555556 0.913
Weighted Loss of LogisticQuantileAT, gamma=0.6666666666666666 1.1523333333333332
0-1 Loss of LogisticQuantileAT, gamma=0.6666666666666666 0.913
Weighted Loss of LogisticQuantileAT, gamma=0.7777777777777778 0.871
0-1 Loss of LogisticQuantileAT, gamma=0.7777777777777778 0.901
Weigh

In [93]:
AT_preds = np.zeros((n, s))
for i in range(1, s):
    AT_preds[:, i] = AT_predictions[i - 1]
AT_preds = AT_preds.astype(int)
quantile_predictions = np.zeros((n, 1))

for i in range(1, n):
    quantiles = AT_preds[i, :]
    quantile_predictions[i, :] = np.random.choice(mode(quantiles.tolist()))
    
metrics.zero_one_loss(quantile_predictions, test_y)

0.902

# Run Multiclass Algorithms (for Comparison)

In [50]:
# Standard Logisitic regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(train_X, train_y)
metrics.zero_one_loss(clf.predict(test_X), test_y)

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.18100000000000005

# Analyze Results

In [50]:
import pandas as pd

In [51]:
results_df = pd.read_csv('mnist_opt/results.csv')

In [53]:
results_df.groupby('Opt_Type').mean()

,Quantile,Kernel_Parameter,Reg_Parameter,Weighted_Loss,inSampleWeighted,Time
Opt_Type,,,,,,
AMSGrad,0.5,1,10,1.473583,1.485885,3962.451075
Momentum,0.5,1,10,0.577954,0.574733,3922.137073
Nesterov_Momentum,0.5,1,10,0.577675,0.574647,3915.625747
SGD,0.5,1,10,0.579679,0.579092,3871.359769


In [54]:
results_df.groupby('Opt_Type').std()

,Quantile,Kernel_Parameter,Reg_Parameter,Weighted_Loss,inSampleWeighted,Time
Opt_Type,,,,,,
AMSGrad,0.223607,0.0,0.0,0.519959,0.528222,28.767612
Momentum,0.223607,0.0,0.0,0.080895,0.079966,36.956752
Nesterov_Momentum,0.223607,0.0,0.0,0.076357,0.076789,39.952209
SGD,0.223607,0.0,0.0,0.079105,0.077479,122.200458
